In [ ]:
import tkinter as tk
import random
import winsound  # For sound effects (Windows only)

# ----------------------------------------
# Modern Pong with AI and Sound Effects
# ----------------------------------------

class PongGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Pong with AI by Lowell")

        # --- Canvas setup ---
        self.canvas = tk.Canvas(root, width=800, height=600, bg="black")
        self.canvas.pack()

        # --- Game objects ---
        self.ball = self.canvas.create_oval(390, 290, 410, 310, fill="white")
        self.paddle_a = self.canvas.create_rectangle(30, 250, 50, 350, fill="cyan")
        self.paddle_b = self.canvas.create_rectangle(750, 250, 770, 350, fill="magenta")

        # --- Score text ---
        self.score_a = 0
        self.score_b = 0
        self.score_text = self.canvas.create_text(
            400, 30, text="Player A: 0   Player B: 0",
            fill="white", font=("Courier", 20)
        )

        # --- Buttons ---
        control_frame = tk.Frame(root, bg="black")
        control_frame.pack(pady=5)

        self.start_button = tk.Button(control_frame, text="Start Game", font=("Courier", 14),
                                      command=self.start_game)
        self.start_button.grid(row=0, column=0, padx=10)

        self.pause_button = tk.Button(control_frame, text="Pause", font=("Courier", 14),
                                      command=self.toggle_pause)
        self.pause_button.grid(row=0, column=1, padx=10)

        self.reset_button = tk.Button(control_frame, text="Restart", font=("Courier", 14),
                                      command=self.reset_game)
        self.reset_button.grid(row=0, column=2, padx=10)

        self.ai_button = tk.Button(control_frame, text="Toggle AI", font=("Courier", 14),
                                   command=self.toggle_ai)
        self.ai_button.grid(row=0, column=3, padx=10)

        # --- Game variables ---
        self.dx = random.choice([-5, 5])
        self.dy = random.choice([-5, 5])
        self.game_running = False
        self.paused = False
        self.ai_enabled = False

        # --- Keyboard bindings ---
        self.root.bind("w", self.paddle_a_up)
        self.root.bind("s", self.paddle_a_down)
        self.root.bind("<Up>", self.paddle_b_up)
        self.root.bind("<Down>", self.paddle_b_down)

        # Start animation loop
        self.move_ball()

    # --- Paddle movement ---
    def paddle_a_up(self, event=None):
        if self.game_running and not self.paused:
            self.canvas.move(self.paddle_a, 0, -20)

    def paddle_a_down(self, event=None):
        if self.game_running and not self.paused:
            self.canvas.move(self.paddle_a, 0, 20)

    def paddle_b_up(self, event=None):
        if not self.ai_enabled and self.game_running and not self.paused:
            self.canvas.move(self.paddle_b, 0, -20)

    def paddle_b_down(self, event=None):
        if not self.ai_enabled and self.game_running and not self.paused:
            self.canvas.move(self.paddle_b, 0, 20)

    # --- Main loop ---
    def move_ball(self):
        if self.game_running and not self.paused:
            self.canvas.move(self.ball, self.dx, self.dy)
            self.check_collisions()

            # Move AI paddle if enabled
            if self.ai_enabled:
                self.move_ai_paddle()

        self.root.after(20, self.move_ball)

    # --- AI movement ---
    def move_ai_paddle(self):
        ball_coords = self.canvas.coords(self.ball)
        paddle_coords = self.canvas.coords(self.paddle_b)
        ball_y = (ball_coords[1] + ball_coords[3]) / 2
        paddle_y = (paddle_coords[1] + paddle_coords[3]) / 2

        # Follow the ball gradually (AI difficulty)
        if paddle_y < ball_y - 10:
            self.canvas.move(self.paddle_b, 0, 8)
        elif paddle_y > ball_y + 10:
            self.canvas.move(self.paddle_b, 0, -8)

    # --- Collision checks ---
    def check_collisions(self):
        ball = self.canvas.coords(self.ball)
        paddle_a = self.canvas.coords(self.paddle_a)
        paddle_b = self.canvas.coords(self.paddle_b)

        # Bounce off top/bottom
        if ball[1] <= 0 or ball[3] >= 600:
            self.dy = -self.dy
            winsound.PlaySound("SystemAsterisk", winsound.SND_ALIAS)

        # Right wall (Player A scores)
        if ball[2] >= 800:
            self.score_a += 1
            self.update_score()
            self.reset_ball(direction=-1)
            winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

        # Left wall (Player B scores)
        if ball[0] <= 0:
            self.score_b += 1
            self.update_score()
            self.reset_ball(direction=1)
            winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

        # Paddle collisions
        # Paddle A
        if (paddle_a[2] >= ball[0] >= paddle_a[0] and
            paddle_a[1] <= (ball[1] + ball[3]) / 2 <= paddle_a[3]):
            self.dx = abs(self.dx)
            self.increase_speed()
            winsound.PlaySound("SystemHand", winsound.SND_ALIAS)

        # Paddle B
        if (paddle_b[0] <= ball[2] <= paddle_b[2] and
            paddle_b[1] <= (ball[1] + ball[3]) / 2 <= paddle_b[3]):
            self.dx = -abs(self.dx)
            self.increase_speed()
            winsound.PlaySound("SystemHand", winsound.SND_ALIAS)

    def increase_speed(self):
        """Increase ball speed slightly each bounce."""
        self.dx *= 1.05
        self.dy *= 1.05

    # --- Game controls ---
    def start_game(self):
        self.game_running = True
        self.paused = False
        self.start_button.config(state="disabled")

    def toggle_pause(self):
        if self.game_running:
            self.paused = not self.paused
            self.pause_button.config(text="Resume" if self.paused else "Pause")

    def reset_game(self):
        self.score_a = 0
        self.score_b = 0
        self.update_score()
        self.reset_ball()
        self.start_button.config(state="normal")
        self.game_running = False
        self.paused = False
        self.pause_button.config(text="Pause")

    def reset_ball(self, direction=None):
        self.canvas.coords(self.ball, 390, 290, 410, 310)
        self.dx = random.choice([-5, 5]) if direction is None else (5 * direction)
        self.dy = random.choice([-5, 5])

    def toggle_ai(self):
        self.ai_enabled = not self.ai_enabled
        self.ai_button.config(text="AI: ON" if self.ai_enabled else "AI: OFF")

    def update_score(self):
        self.canvas.itemconfig(
            self.score_text,
            text=f"Player A: {self.score_a}   Player B: {self.score_b}"
        )

# --- Run the game ---
if __name__ == "__main__":
    root = tk.Tk()
    game = PongGame(root)
    root.mainloop()
